In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os


#parameters for the lane detection
#k_size"        Size of Gaussian Kernel for Gaussian smoothing step
#low_thres"     Lower threshold value for Gaussian smoothing
#hi_thres"      Higher threshold value for Gaussian smoothing
#rho"           Radius value for Hough Transform
#theta"         Theta for Hough Transform
#hough_thres    Hough threshold parameter
#min_len"       Minimum length of lines found by HT
#max_gap"       Maximum gap allowed for lines detectde by HT
#alpha"         Weight for original image
#gamma"         Weight for masked image
#beta"          Scalar to be added
params = { "k_size"     : 3  \
          ,"low_thres"  : 200 \
          ,"hi_thres"   : 400\
          ,"rho"        : 1  \
          ,"theta"      : 1*np.pi/180  \
          ,"hough_thres": [100, 10] \
          ,"min_len"    : [50, 20] \
          ,"max_gap"    : [25, 50]  \
          ,"alpha"      : 0.85\
          ,"gamma"      : 0.0\
          ,"beta"       : 1.0\
             }  

#define color ranges for yellow (HSV) and white(RGB)
yellow_bd = [([95, 50, 50], [100, 255, 255])]
white_bd = [([200, 200, 200], [255, 255, 255])]

def get_vertices(imgSize, isChallenge):
    """Get vertices for ROI"""
    if ( isChallenge ):
        vertices = np.array([[(220,imgSize-40),(585, 455), (730, 455), (1100,imgSize-40)]], dtype=np.int32)        
    else:
        vertices = np.array([[(90,imgSize),(430, 330), (520, 330), (900,imgSize)]], dtype=np.int32)
    return vertices

def color_of_interest(image, color):
    """Keeps only the color of interest
    This will try to detect only the color specified
    according to the range given (color = (lower, upper)"""
    for lower, upper in color:
        lower = np.array(lower, dtype="uint8")
        upper = np.array(upper, dtype="uint8")
        mask = cv2.inRange(image, lower, upper)
    return cv2.bitwise_and(image, image, mask=mask)


def grayscale(img):
    """Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')"""
    #return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    
    return masked_image


def draw_lines(img, lines, color=[0, 0, 255], thickness=1):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap, color, thickness, draw=False):
    """
    `img` should be the output of a Canny transform.
        
    Returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    if (draw):
        line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
        draw_lines(line_img, lines, color, thickness)
        return lines, line_img
    return lines

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, alpha=0.8, beta=1., gamma=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * �� + img * �� + ��
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, alpha, img, beta, gamma)

def prep_image(image, ksize, isChallenge, debug=False):
    """Applies the following to the original image:
       Isolate colors of interest (yellow and white)
       Grayscale the image
       Gaussian Noise kernel
       Increase contrast
       """
    # Convert to HSV
    img_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Isolate colors of interest
    img_yellow = color_of_interest(img_hsv, yellow_bd)
    img_white = color_of_interest(image, white_bd)
    mask = img_yellow + img_white
    
    # Duplicate image. Apply mask
    img_color_mask = image.copy()
    img_color_mask[np.where(mask==0)] = 0
    img_color_mask[np.where(mask!=0)] = 255
    
    # RGB -> Grayscale
    img_gray = grayscale(img_color_mask)
    
    # Grayscaled -> Gaussian Blur Grayscale
    img_gray_gb = gaussian_blur(img_gray, ksize)
    
    # Increase contrast
    img_contrast = cv2.multiply(img_gray_gb,1.2)
    
    if (False):
        img1 = cv2.resize(img_yellow, (int(image.shape[0]/2), int(image.shape[1]/4)))
        img2 = cv2.resize(img_white, (int(image.shape[0]/2), int(image.shape[1]/4)))
        image[0:img1.shape[0], 0:img1.shape[1]] = img1
        image[0:img2.shape[0], img1.shape[1]:img1.shape[1]+img2.shape[1]] = img2

    return img_contrast


def averageLines(lines, ybottom, ytop, tol=0.5):
    """From a number of line segments, 
       choose lines with slopes similar to lane lines
       Fit a single line between all collected points
       Returns two lines that represent the left and 
       right lanes"""
    # Process lines
    x_m_neg = []
    y_m_neg = []
    x_m_pos = []
    y_m_pos = []
    
    for i in range(0,lines.shape[0]):
        if ( lines[i][0][2] - lines[i][0][0] != 0):
            m = (lines[i][0][3] - lines[i][0][1])/(lines[i][0][2] - lines[i][0][0])
            if ( m>tol ):
                x_m_pos.append(lines[i][0][2])
                x_m_pos.append(lines[i][0][0])
                y_m_pos.append(lines[i][0][3])
                y_m_pos.append(lines[i][0][1])
            elif ( m<-tol ):
                x_m_neg.append(lines[i][0][2])
                x_m_neg.append(lines[i][0][0])
                y_m_neg.append(lines[i][0][3])
                y_m_neg.append(lines[i][0][1])
    lines_left = np.array([[0,0,0,0]])
    lines_right = np.array([[0,0,0,0]])
    
    if (len(x_m_pos)>0 and len(y_m_pos)>0):
        x_pos = np.asarray(x_m_pos)
        y_pos = np.asarray(y_m_pos)
        z_pos = np.polyfit(x_pos, y_pos, 1)
        #intersection with top of image: y = ytop
        y2 = ytop
        x2 = int((y2 - z_pos[1])/z_pos[0])
        #intersection with bottom of image: y = ybottom
        y1 = ybottom
        x1 = int((y1 - z_pos[1])/z_pos[0])
        lines_right = np.array([[x1, y1, x2, y2]])
    
    if (len(x_m_neg)>0 and len(y_m_neg)>0):
        x_neg = np.asarray(x_m_neg)
        y_neg = np.asarray(y_m_neg)
        z_neg = np.polyfit(x_neg, y_neg, 1)
        #intersection with top of image: y = ytop
        y2 = ytop
        x2 = int((y2 - z_neg[1])/z_neg[0])
        #intersection with bottom of image: y = ybottom
        y1 = ybottom
        x1 = int((y1 - z_neg[1])/z_neg[0])
        lines_left = np.array([[x1, y1, x2, y2]])
        
    lines_final = np.array([lines_left, lines_right])
    return lines_final


def hough_pass(image, params, y, tol, debug=False):
    """Apply 2 Hough Transforms with different parameters 
       to capture maximum amount of line data
       Return an empty image with lane lines drawn on it"""
    xdim = image.shape[0]
    ydim = image.shape[1]
    
    # First Hough pass
    lines_p1 = hough_lines(image, params["rho"], \
                                  params["theta"], \
                                  params["hough_thres"][0], \
                                  params["min_len"][0], \
                                  params["max_gap"][0], \
                                  [255,255,255], \
                                  1)
    
    if lines_p1 is not None:
        # Subtract the major line information from main image
        img_fatlines = np.zeros((xdim, ydim, 3), dtype=np.uint8)
        draw_lines(img_fatlines, lines_p1, [255, 255, 255], 30)
        if (debug):
            cv2.imshow("hough_image_pass1", img_fatlines) 
            cv2.waitKey(0)    
        hg_fatlines = grayscale(img_fatlines)
        img_thinlines = cv2.subtract(image, hg_fatlines)
        if (debug):
            cv2.imshow("image_remainder_boosted", img_thinlines) 
            cv2.waitKey(0)    
    else:
        img_thinlines = image
    
    # Second Hough pass
    lines_p2 = hough_lines(img_thinlines, params["rho"], \
                                        params["theta"], \
                                        params["hough_thres"][1], \
                                        params["min_len"][1], \
                                        params["max_gap"][1],  \
                                        [255,255,255], \
                                        1)
    if lines_p2 is not None:
        if (debug):
            img_fatlines = np.zeros((xdim, ydim, 3), dtype=np.uint8)
            draw_lines(img_fatlines, lines_p2, [255, 255, 255], 30)
            cv2.imshow("hough_image_pass2", img_fatlines) #, cmap='gray')
            cv2.waitKey(0)

    if (lines_p1 is not None and lines_p2 is not None):
        lines_all = np.concatenate((lines_p1, lines_p2), axis=0)
    elif (lines_p2 is None and lines_p1 is not None):
        lines_all = lines_p1.copy()
    elif (lines_p1 is None and lines_p2 is not None):
        lines_all = lines_p2.copy()
    else:
        lines_all = np.empty()
    # Get a set of two lines for both lanes
    lines_final = averageLines(lines_all, y[0], y[1], tol)
    
    # Draw lines on empty image and return
    img_cleanlines = np.zeros((xdim, ydim, 3), dtype=np.uint8)
    draw_lines(img_cleanlines, lines_final, [0, 255, 0], 7)
    if (debug):
        cv2.imshow("lines_drawn", img_cleanlines) #, cmap='gray')
        cv2.waitKey(0)    
    
    return img_cleanlines


def pipeline_LD(image, debug=False, isChallenge=False):
    """Prep image, detect edges, find lane lines
       Return original image with lane lines drawn on it
       Use debug=True for additional information"""
    
    x_y_img = image.shape
    
    if (debug):
        cv2.imshow("original_image", image)
        cv2.waitKey(0)

    # Prep the image (brightness, gauss blur)
    img_prep = prep_image(image, params["k_size"], isChallenge, True)
    if (debug):
        cv2.imshow("prepped_image", img_prep) 
        cv2.waitKey(0)

    # Canny edge detection
    img_cny = canny(img_prep, params["low_thres"], params["hi_thres"])
    if (debug):
        cv2.imshow("canny_edge", img_cny) 
        cv2.waitKey(0)
    
    # Create masked image for ROI
    vertices = get_vertices(x_y_img[0], isChallenge)
    img_masked = region_of_interest(img_cny, vertices) 
    if (debug):
        cv2.imshow("img_masked", img_masked)
        cv2.waitKey(0)
        
    # Image with Hough lines (double_pass)
    slope_tolerance = 0.5
    img_hough = hough_pass(img_masked, params, [vertices[0][0][1], \
                            vertices[0][1][1]],slope_tolerance)    
    
    # Overlay with original image
    img_lanes = weighted_img(img_hough, image, params["alpha"], \
                                            params["beta"], \
                                            params["gamma"])    
        
    if (debug):
        lines_vertices = np.array([[[vertices[0][0][0],vertices[0][0][1],vertices[0][1][0],vertices[0][1][1]],
                                    [vertices[0][1][0],vertices[0][1][1],vertices[0][2][0],vertices[0][2][1]],
                                    [vertices[0][2][0],vertices[0][2][1],vertices[0][3][0],vertices[0][3][1]],
                                    [vertices[0][3][0],vertices[0][3][1],vertices[0][0][0],vertices[0][0][1]]]])
        draw_lines(img_lanes, lines_vertices, [0,255,0], 1)
        cv2.imshow("lane_detected",img_lanes)
        cv2.waitKey(0)        
    
    return img_lanes

def process_image(image):
    result = pipeline_LD(image, False, False)
    return result

def process_image_challenge(image):
    result = pipeline_LD(image, False, True)
    return result

from moviepy.editor import VideoFileClip
from IPython.display import HTML
vid_output = ['yellow.mp4','white.mp4','challenge_lanes.mp4']
vid_input = ['solidYellowLeft.mp4','solidWhiteRight.mp4','challenge.mp4']

for i in range(0,len(vid_input)):
    vid_clip = VideoFileClip(vid_input[i])
    if ( i == 2 ):
        vid_clip = vid_clip.fl_image(process_image_challenge)
    else:
        vid_clip = vid_clip.fl_image(process_image)
    vid_clip.write_videofile(vid_output[i], audio=False)

[MoviePy] >>>> Building video yellow.mp4
[MoviePy] Writing video yellow.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:42<00:00, 16.02it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: yellow.mp4 

[MoviePy] >>>> Building video white.mp4
[MoviePy] Writing video white.mp4


100%|███████████████████████████████████████████████████████████████████████████████▋| 221/222 [00:14<00:00, 14.88it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: white.mp4 

[MoviePy] >>>> Building video challenge_lanes.mp4
[MoviePy] Writing video challenge_lanes.mp4


100%|████████████████████████████████████████████████████████████████████████████████| 251/251 [00:30<00:00,  8.62it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: challenge_lanes.mp4 

